# TrOCR: Transformer-based Optical Character Recognition with Pre-trained Models


The author proposes an end-to-end transformer-based OCR model for text recognition with pre-trained CV and NLP models.

One benefit of TrOCR is that it:

- TrOCR uses image transformer and text transformer models that have already been trained. These models use large amounts of unstructured data to understand images and model language, so they don't need an external language model.

- TrOCR doesn't need any inductive biases or convolutional networks, which makes the model very simple to set up and keep up to date.
- Based on the results of the experiment on the OCR benchmark dataset, TrOCR gets top-notch results on printed, handwritten, and scene text image datasets with no complicated pre- or post-processing steps. It can also be easily expanded to recognize text in multiple languages.

# Model Architecture

For the architecture, the authors adopt the vanilla transformer encoder-decoder structure in TrOCR. The encoder is designed to obtain the representation of the input image patches and the decoder is aimed at generating the sequence of words guided by the visual features and previous predictions.

**Encoder** 

After receiving an input image, the encoder resizes it to a predetermined size. Since the encoder does not process raw images, the input picture must be converted into tokens. After splitting the input picture into PxP patches, the encoder flattens the patches to create vectors, which are then projected onto a linear transformation to create embedding vectors.

According to the author, they retain the unique tokens "[CLS]," which are used for image classification tasks. The "[CLS]" represents the entire image by compiling all of the data from all of the patch embeddings. Additionally, when they use DeiT pre-trained models for encoder initialization, they retain the distillation token in the input sequence.

what separates TrOCR from traditional CNN-like networks is that it has no inductive biases and process the image as a sequence of patches, which enables the model to pay attention to different patches and the whole image easily.

**Decoder**

The original transformer decoder, which is the standard transformer decoder, is used by the authors for the decoder. It also consists of a stack of identical layers with structures similar to those of the encoder, with the exception that the decoder adds "encoder-decoder attention" between the feed-forward network and multi-head self attention to allocate distinct attention to the encoder's output.

In the encoder-decoder attention module, the keys and values come from the encoder while the queries come from the decoder input. Moreover, the decoder adopt the attention masking in the self-attention to prevent itself from getting more information during training than prediction.

# Model Initialization

**Encoder Initialization**

The encoder of TrOCR are constructed from a combination of two models, DeiT and BEiT. DEiT is trained with ImageNet which is one of the biggest image datasets in  the world. Various hyperparameters and data augmentation techniques were used for the training. In addition, the authers of DEiT also performed knowlede distillation from a strong image classifier to a distilled token in the initial embedding, which enhance the competitiveness of the result compared to the CNN-based models.

On the other hand, BEiT used the Masked Image Modeling task to pre-train the image transformer. The technique randomly masks some image patches, and enforces the model to reconstruct the masked patches to form the original image.

**Decoder Initialization**

The MiniLM and RoBERTa models are used for decoder initialization. The influence of several important hyperparameters and training data size is meticulously measured in RoBERTa, which is only a replication study of Devlin et al. 2019. Similar to BERT, the inventors of RoBERTa changed the Masked Language Model's masking pattern dynamically and eliminated the next sentence prediction aim. 

The MiniLM are compressed models of the large pretrained Transformer models but retaining 99% performance. Putting together the two models to the decoder, the structure is not perfectly aligned since both of the models are encoder, meaning that some layers , for example, encoder-decoder attention is absent. The author addressed this issue by manually setting the corresponding parameter mapping, and randomly initialized the absent parameters